In [11]:
import os
import requests
import pandas as pd
import geopandas as gpd
import osmnx as ox
from bs4 import BeautifulSoup
from io import BytesIO, StringIO
from zipfile import ZipFile
from pathlib import Path
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import matplotlib.pyplot as plt
import contextily as ctx
import mapclassify
import panel as pn
import hvplot.pandas
import numpy as np
import warnings
import networkx as nx
from matplotlib_scalebar.scalebar import ScaleBar

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

In [3]:
# Base directory of your project
BASE_DIR = Path(".")

# Census and geography constants
PHILLY_STATE_FIPS = "42"
PHILLY_COUNTY_FIPS = "101"
# Data directories
DATA_DIR = BASE_DIR / "data"
DATA_RAW = DATA_DIR / "raw"
DATA_PROCESSED = DATA_DIR / "processed"

DATA_PROCESSED.mkdir(parents=True, exist_ok=True)

# Headers to mimic a browser
HEADERS = {
    "User-Agent": (
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/91.0.4472.124 Safari/537.36"
    )
}
G = ox.graph_from_place("Philadelphia, Pennsylvania, USA", network_type='drive')
ox.save_graph_shapefile(G, filepath="./Data/philadelphia_osm_roads")

In [ ]:
api_key = "02cfb7601cb4d601454bee8c28d14d90211b5baf"
url = ("https://api.census.gov/data/2021/acs/acs5?get=B01003_001E"
       "&for=block%20group:*&in=state:42%20county:101&key=" + api_key)
resp = requests.get(url)
data = resp.json() 
columns, *rows = data
df = pd.DataFrame(rows, columns=columns)
df.rename(columns={"B01003_001E": "population"}, inplace=True)
print(df.head())

In [15]:
def fetch_hospitals():
    """Fetches Philadelphia Hospitals from OpenDataPhilly (GeoJSON)."""
    print("🏥 Fetching Philadelphia Hospitals...")
    # Updated: Using a direct stable resource URL found on OpenDataPhilly
    url = "https://opendata.arcgis.com/datasets/df8dc18412494e5abbb021e2f33057b2_0.geojson"
    
    try:
        response = requests.get(url, headers=HEADERS)
        if response.status_code == 200:
            output_path = DATA_DIR / "phl_hospitals.geojson"
            with open(output_path, 'wb') as f:
                f.write(response.content)
            print(f"   ✅ Saved hospitals to {output_path}")
        else:
            print(f"   ❌ API Error: {response.status_code}")
    except Exception as e:
        print(f"   ❌ Failed to fetch hospitals: {e}")

def scrape_urgent_care_backup():
    """Backup: Scrapes address list if GeoJSON fails."""
    print("🕸️  (Backup) Scraping Health Centers from Web...")
    url = "https://www.phila.gov/services/mental-physical-health/city-health-centers/"
    
    try:
        response = requests.get(url, headers=HEADERS)
        soup = BeautifulSoup(response.content, 'html.parser')
        centers = []
        tables = soup.find_all("table")
        
        for table in tables:
            rows = table.find_all("tr")[1:]
            for row in rows:
                cols = row.find_all("td")
                if len(cols) >= 2:
                    name = cols[0].get_text(strip=True)
                    address = cols[1].get_text(strip=True).replace('\n', ' ')
                    centers.append({"name": name, "address": address, "type": "City Health Center"})
        
        if centers:
            df = pd.DataFrame(centers)
            csv_path = DATA_DIR / "scraped_health_centers.csv"
            df.to_csv(csv_path, index=False, encoding='utf-8')
            print(f"   ✅ Scraped {len(df)} centers to {csv_path}")
        else:
            print("   ⚠️  No table data found on page.")
            
    except Exception as e:
        print(f"   ❌ Failed to scrape: {e}")

In [16]:
def fetch_census_tracts():
    """Fetches PA Census Tracts and filters for Philly."""
    print("🗺️  Fetching Philadelphia Census Tracts...")
    # PA State FIPS = 42
    url = "https://www2.census.gov/geo/tiger/TIGER2021/TRACT/tl_2021_42_tract.zip"
    
    try:
        r = requests.get(url, headers=HEADERS)
        if r.status_code == 200:
            with ZipFile(BytesIO(r.content)) as z:
                z.extractall(DATA_DIR / "temp_tracts")
            
            # Read and filter
            shp_path = DATA_DIR / "temp_tracts" / "tl_2021_42_tract.shp"
            gdf = gpd.read_file(shp_path)
            # Filter for Philly (County Code 101)
            philly_tracts = gdf[gdf['COUNTYFP'] == '101']
            
            output_path = DATA_DIR / "philly_tracts.geojson"
            philly_tracts.to_file(output_path, driver="GeoJSON")
            print(f"   ✅ Saved {len(philly_tracts)} tracts to {output_path}")
        else:
            print(f"   ❌ API Error: {r.status_code}")
    except Exception as e:
        print(f"   ❌ Failed to fetch tracts: {e}")

def fetch_hud_qct():
    """Fetches HUD QCT via ArcGIS FeatureServer."""
    print("🏘️  Fetching HUD Qualified Census Tracts (QCT)...")
    # Corrected URL for the HUD Feature Service
    url = "https://services.arcgis.com/g1fRTDLeMgspWrYp/arcgis/rest/services/QCTs/FeatureServer/0/query"
    
    params = {
        'where': "STATE='42' AND COUNTY='101'", # PA, Philly
        'outFields': '*',
        'f': 'geojson'
    }
    
    try:
        r = requests.get(url, params=params, headers=HEADERS)
        if r.status_code == 200:
            data = r.json()
            # Check if features exist
            if 'features' in data and len(data['features']) > 0:
                gdf = gpd.GeoDataFrame.from_features(data['features'])
                output_path = DATA_DIR / "hud_qct.geojson"
                gdf.to_file(output_path, driver="GeoJSON")
                print(f"   ✅ Saved {len(gdf)} QCT records to {output_path}")
            else:
                print("   ⚠️  No QCT records returned. Check query parameters.")
        else:
            print(f"   ❌ API Error: {r.status_code}")
    except Exception as e:
        print(f"   ❌ Failed to fetch HUD QCT: {e}")

In [17]:
def fetch_road_network():
    """Fetches drive network."""
    print("🛣️  Downloading Road Network...")
    output_path = DATA_DIR / "philly_drive_network.graphml"
    
    # Check if it already exists to save time
    if output_path.exists():
        print(f"   ✅ Network already exists at {output_path}. Skipping download.")
        return

    try:
        G = ox.graph_from_place("Philadelphia, PA", network_type='drive')
        ox.save_graphml(G, output_path)
        print(f"   ✅ Saved road network to {output_path}")
    except Exception as e:
        print(f"   ❌ Failed to fetch network: {e}")

In [ ]:
if __name__ == "__main__":
    print(f"🚀 STARTING PIPELINE V2")
    print(f"📂 Data Directory: {DATA_DIR}\n")
    
    fetch_hospitals()
    scrape_urgent_care_backup()
    fetch_census_tracts()
    fetch_hud_qct()
    fetch_road_network()
    
    print("\n🎉 DONE! Check your 'data/raw' folder.")

In [18]:
# 1. Environment Settings
warnings.filterwarnings('ignore')
pn.extension() # Initialize Panel for interactive plots

# 2. Path Configuration
# Use relative path or your specific absolute path
BASE_DIR = Path(r".") 
DATA_PROCESSED = BASE_DIR / "data" / "processed"
RESULTS_DIR = BASE_DIR / "results"

# 3. Create Specific Output Directories
DIRS = {
    "error": RESULTS_DIR / "01_missteps_edge_effects",
    "process": RESULTS_DIR / "02_intermediate_process",
    "final": RESULTS_DIR / "03_final_deliverables"
}

for d in DIRS.values():
    d.mkdir(parents=True, exist_ok=True)

# 4. Coordinate Reference System (CRS) Settings
# User requested EPSG:2272, but also requested "Meters".
# EPSG:2272 is PA South (US Feet). 
# EPSG:32129 is PA South (Meters). 
# We use 32129 to satisfy the "Meters" requirement for scientific accuracy.
CRS_METRIC = "EPSG:32129" 
CRS_VISUAL = "EPSG:3857"   # Web Mercator (Required for Contextily Basemaps)

# 5. Plotting Aesthetics
plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['figure.dpi'] = 300 # High resolution for publication

print(f"✅ Environment Configured.")
print(f"✅ Projection Set: {CRS_METRIC} (Pennsylvania South - Meters)")
print(f"📂 Output Folders Ready: {[d.name for d in DIRS.values()]}")

✅ Environment Configured.
✅ Projection Set: EPSG:32129 (Pennsylvania South - Meters)
📂 Output Folders Ready: ['01_missteps_edge_effects', '02_intermediate_process', '03_final_deliverables']


In [19]:
import geopandas as gpd
import rasterio
from rasterio.mask import mask
from rasterstats import zonal_stats
from shapely.geometry import mapping
from pathlib import Path
import warnings

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

# Configuration (Ensure these match your project structure)
BASE_DIR = Path(".")
DATA_DIR = BASE_DIR / "data"
DATA_RAW = DATA_DIR / "raw"
DATA_PROCESSED = DATA_DIR / "processed"

# Ensure directories exist
DATA_PROCESSED.mkdir(parents=True, exist_ok=True)

def process_worldpop_raster():
    """
    Task 2: Clip WorldPop Raster and Calculate Zonal Statistics (Fixed Version)
    """
    print("🌍 Processing WorldPop Raster...")
    
    # Define paths
    # Ensure the filename matches exactly what you downloaded
    raster_path = DATA_RAW / "usa_pop_2020_CN_100m_R2025A_v1.tif" 
    tracts_path = DATA_PROCESSED / "tracts_svi_projected.geojson"
    
    if not raster_path.exists():
        print(f"   ⚠️ WorldPop file not found at {raster_path}. Skipping.")
        return

    # 1. Read Boundary Data (Tracts) and Convert to WGS84
    # WorldPop data is usually in EPSG:4326 (Lat/Lon), so we match the vector data to the raster.
    gdf = gpd.read_file(tracts_path)
    gdf_wgs = gdf.to_crs(epsg=4326) 
    
    # [CRITICAL FIX] Create a unified boundary for clipping
    # Use mapping() to convert the Shapely geometry object into the GeoJSON format required by Rasterio
    union_geom = gdf_wgs.unary_union
    philly_shape = [mapping(union_geom)] 
    
    try:
        # 2. Clip the Raster
        print("   ...Clipping raster to Philadelphia boundary...")
        with rasterio.open(raster_path) as src:
            out_image, out_transform = mask(src, philly_shape, crop=True)
            out_meta = src.meta.copy()
            
        # Update metadata for the clipped file
        out_meta.update({
            "driver": "GTiff",
            "height": out_image.shape[1],
            "width": out_image.shape[2],
            "transform": out_transform
        })
        
        # Save Clipped Raster
        clipped_path = DATA_PROCESSED / "philly_pop_clipped.tif"
        with rasterio.open(clipped_path, "w", **out_meta) as dest:
            dest.write(out_image)
        print(f"   ✅ Clipped raster saved to: {clipped_path}")
        
        # 3. Calculate Zonal Statistics
        # Sum pixel values within each tract polygon to estimate population
        print("   ...Calculating Zonal Statistics...")
        stats = zonal_stats(gdf_wgs, clipped_path, stats="sum")
        
        # Assign results back to the GeoDataFrame
        gdf['worldpop_sum'] = [s['sum'] for s in stats]
        
        # Save Comparison Data
        output_path = DATA_PROCESSED / "tracts_census_vs_worldpop.geojson"
        gdf.to_file(output_path, driver="GeoJSON")
        print(f"   ✅ Zonal stats calculated. Saved comparison data to: {output_path}")
        
    except Exception as e:
        print(f"   ❌ Raster processing failed: {e}")

if __name__ == "__main__":
    # If you have the census function defined elsewhere, you can call it here:
    # fetch_and_merge_census_pop()
    process_worldpop_raster()

🌍 Processing WorldPop Raster...
   ...Clipping raster to Philadelphia boundary...
   ✅ Clipped raster saved to: data\processed\philly_pop_clipped.tif
   ...Calculating Zonal Statistics...
   ✅ Zonal stats calculated. Saved comparison data to: data\processed\tracts_census_vs_worldpop.geojson


In [20]:
CENSUS_API_KEY = "02cfb7601cb4d601454bee8c28d14d90211b5baf" 

TARGET_CRS = "EPSG:32129" # PA South (Meters)

def fetch_and_merge_census_pop():
    """
    Task 1: Fetch Block Group population from Census API, aggregate, and merge to Tracts.
    """
    print("👥 Fetching Real Population from Census API...")
    
    # 1. API Request (2021 ACS 5-Year, Variable B01003_001E = Total Population)
    # We request block group data to ensure high granularity, then aggregate up.
    url = f"https://api.census.gov/data/2021/acs/acs5?get=B01003_001E&for=block%20group:*&in=state:42%20county:101&key={CENSUS_API_KEY}"
    
    try:
        resp = requests.get(url)
        if resp.status_code != 200:
            raise Exception(f"API Error: {resp.status_code} {resp.text}")
            
        data = resp.json()
        # Create DataFrame (skip the first row which is the header)
        df = pd.DataFrame(data[1:], columns=data[0]) 
        
        # 2. Data Cleaning
        df['population'] = df['B01003_001E'].astype(int)
        
        # Construct GEOID: state(2) + county(3) + tract(6)
        # We need to aggregate to Tract level (first 11 digits) from Block Groups
        df['tract_geoid'] = df['state'] + df['county'] + df['tract']
        
        # 3. Groupby Aggregation (Summing Block Groups into Tracts)
        df_tract_pop = df.groupby('tract_geoid')['population'].sum().reset_index()
        print(f"   ✅ Aggregated population for {len(df_tract_pop)} tracts.")
        
        # 4. Merge into existing Tracts GeoJSON
        tracts_path = DATA_PROCESSED / "tracts_svi_projected.geojson"
        
        if not tracts_path.exists():
            print(f"   ❌ Error: File not found at {tracts_path}")
            return None

        gdf_tracts = gpd.read_file(tracts_path)
        
        # Ensure Key types match for merging
        # Identify the correct column for GEOID in the shapefile
        geoid_col = 'GEOID' if 'GEOID' in gdf_tracts.columns else 'GEOID20'
        
        # If the file was merged with SVI previously, it might use 'FIPS'
        if 'FIPS' in gdf_tracts.columns:
            left_key = 'FIPS'
        else:
            left_key = geoid_col
            
        # Convert to string to ensure matching works
        gdf_tracts[left_key] = gdf_tracts[left_key].astype(str)
        
        # Merge
        gdf_final = gdf_tracts.merge(df_tract_pop, left_on=left_key, right_on='tract_geoid', how='left')
        
        # Fill missing values (just in case some tracts have no population data)
        gdf_final['population'] = gdf_final['population'].fillna(0)
        
        # Save
        out_path = DATA_PROCESSED / "tracts_pop_svi_projected.geojson"
        gdf_final.to_file(out_path, driver="GeoJSON")
        print(f"   ✅ Saved updated tracts with REAL population to: {out_path}")
        return gdf_final
        
    except Exception as e:
        print(f"   ❌ Failed to fetch census population: {e}")
        return None

if __name__ == "__main__":
    fetch_and_merge_census_pop()

👥 Fetching Real Population from Census API...
   ✅ Aggregated population for 408 tracts.
   ✅ Saved updated tracts with REAL population to: data\processed\tracts_pop_svi_projected.geojson


In [21]:
TARGET_CRS = "EPSG:32129"

def process_supply_data():
    print("🏥 Processing Supply Data (Hospitals & Health Centers)...")
    
    # 1. Process official hospital data (GeoJSON)
    hosp_path = DATA_RAW / "phl_hospitals.geojson"
    if hosp_path.exists():
        gdf_hosp = gpd.read_file(hosp_path)
        # Keep only useful columns
        cols = ['HOSPITAL_NAME', 'HOSPITAL_TYPE', 'BEDS', 'geometry']
        # Adjust if column names differ. Try automatic matching of common names.
        available_cols = [c for c in cols if c in gdf_hosp.columns]
        gdf_hosp = gdf_hosp[available_cols]
        gdf_hosp['facility_type'] = 'Hospital'
        gdf_hosp = gdf_hosp.to_crs(TARGET_CRS)
    else:
        print("   ⚠️ Missing Hospital Data!")
        gdf_hosp = gpd.GeoDataFrame()

    # 2. Process scraped health centers (CSV -> Geocoding)
    center_path = DATA_RAW / "scraped_health_centers.csv"
    if center_path.exists():
        df_centers = pd.read_csv(center_path)
        print(f"   📍 Geocoding {len(df_centers)} scraped addresses...")
        
        # Initialize geocoder
        geolocator = Nominatim(user_agent="philly_health_access_study")
        geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)
        
        # Execute conversion
        df_centers['location'] = df_centers['address'].apply(geocode)
        df_centers['point'] = df_centers['location'].apply(lambda loc: tuple(loc.point) if loc else None)
        
        # Remove failed entries
        df_centers = df_centers.dropna(subset=['point'])
        
        # Create GeoDataFrame
        # Point format is usually (lat, lon, altitude), we need (lon, lat)
        df_centers['geometry'] = df_centers['point'].apply(lambda x: gpd.points_from_xy([x[1]], [x[0]])[0])
        gdf_centers = gpd.GeoDataFrame(df_centers, geometry='geometry', crs="EPSG:4326")
        
        gdf_centers['facility_type'] = 'City Health Center'
        # Unify projection
        gdf_centers = gdf_centers.to_crs(TARGET_CRS)
        
        # Merge hospitals and health centers
        gdf_facilities = pd.concat([gdf_hosp, gdf_centers], ignore_index=True)
        
        # Save
        out_path = DATA_PROCESSED / "facilities_projected.geojson"
        gdf_facilities.to_file(out_path, driver="GeoJSON")
        print(f"   ✅ Saved merged facilities to {out_path}")
    else:
        print("   ⚠️ Missing Scraped Center Data!")

def process_demand_data():
    print("map Processing Demand Data (Tracts & SVI)...")
    
    # 1. Read Census Tracts
    tract_path = DATA_RAW / "philly_tracts.geojson"
    if not tract_path.exists():
        print("   ❌ Tract data missing.")
        return
    
    gdf_tracts = gpd.read_file(tract_path)
    
    # Ensure GEOID is string format for merging
    # Column name might be 'GEOID' or 'GEOID20' depending on the year
    geoid_col = 'GEOID' if 'GEOID' in gdf_tracts.columns else 'GEOID20'
    gdf_tracts[geoid_col] = gdf_tracts[geoid_col].astype(str)
    
    # 2. Read SVI Data (CSV)
    svi_path = DATA_RAW / "philly_svi_2020.csv"
    if svi_path.exists():
        df_svi = pd.read_csv(svi_path)
        
        # SVI FIPS column is usually 'FIPS'. If numeric, convert to string and pad with zeros.
        # Philly FIPS starts with 42101, length should be 11 digits.
        df_svi['FIPS'] = df_svi['FIPS'].astype(str).str.zfill(11)
        
        # Select key indicators (RPL_THEMES is the overall SVI index)
        # RPL_THEME1: Socioeconomic, RPL_THEME2: Household Comp, etc.
        svi_cols = ['FIPS', 'RPL_THEMES', 'RPL_THEME1', 'RPL_THEME2', 'RPL_THEME3', 'RPL_THEME4']
        df_svi_clean = df_svi[svi_cols].copy()
        
        # 3. Merge data
        gdf_merged = gdf_tracts.merge(df_svi_clean, left_on=geoid_col, right_on='FIPS', how='left')
        
        # Fill missing values (just in case)
        gdf_merged['RPL_THEMES'] = gdf_merged['RPL_THEMES'].fillna(-999) # -999 indicates missing
        
        # Unify projection
        gdf_merged = gdf_merged.to_crs(TARGET_CRS)
        
        # Save
        out_path = DATA_PROCESSED / "tracts_svi_projected.geojson"
        gdf_merged.to_file(out_path, driver="GeoJSON")
        print(f"   ✅ Saved SVI-enriched tracts to {out_path}")
        
    else:
        print("   ⚠️ SVI CSV missing! Please download it manually as instructed.")

def process_qct_data():
    print("🏘️  Processing HUD QCT...")
    qct_path = DATA_RAW / "hud_qct.geojson"
    if qct_path.exists():
        gdf_qct = gpd.read_file(qct_path)
        gdf_qct = gdf_qct.to_crs(TARGET_CRS)
        
        out_path = DATA_PROCESSED / "qct_projected.geojson"
        gdf_qct.to_file(out_path, driver="GeoJSON")
        print(f"   ✅ Saved projected QCT to {out_path}")

if __name__ == "__main__":
    print("🚀 STARTING DATA PREPROCESSING...")
    process_supply_data()
    process_demand_data()
    process_qct_data()
    print("\n🎉 Preprocessing Complete! Ready for Analysis.")

🚀 STARTING DATA PREPROCESSING...
🏥 Processing Supply Data (Hospitals & Health Centers)...
   📍 Geocoding 9 scraped addresses...


RateLimiter caught an error, retrying (0/2 tries). Called with (*('1930 S Broad St., Fl. 2Philadelphia, PA 19145',), **{}).
Traceback (most recent call last):
  File "D:\Geospatial_Workspace\geo310\lib\site-packages\geopy\geocoders\base.py", line 368, in _call_geocoder
    result = self.adapter.get_json(url, timeout=timeout, headers=req_headers)
  File "D:\Geospatial_Workspace\geo310\lib\site-packages\geopy\adapters.py", line 472, in get_json
    resp = self._request(url, timeout=timeout, headers=headers)
  File "D:\Geospatial_Workspace\geo310\lib\site-packages\geopy\adapters.py", line 500, in _request
    raise AdapterHTTPError(
geopy.adapters.AdapterHTTPError: Non-successful status code 403

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "D:\Geospatial_Workspace\geo310\lib\site-packages\geopy\extra\rate_limiter.py", line 136, in _retries_gen
    yield i  # Run the function.
  File "D:\Geospatial_Workspace\geo310\lib\sit

   ✅ Saved merged facilities to data\processed\facilities_projected.geojson
map Processing Demand Data (Tracts & SVI)...
   ✅ Saved SVI-enriched tracts to data\processed\tracts_svi_projected.geojson
🏘️  Processing HUD QCT...
   ✅ Saved projected QCT to data\processed\qct_projected.geojson

🎉 Preprocessing Complete! Ready for Analysis.


In [22]:
from difflib import get_close_matches
CMS_FILE = DATA_RAW / "cms_hospitals.csv"
FACILITIES_FILE = DATA_PROCESSED / "facilities_with_buffer.geojson" 

def enrich_facilities_with_cms():
    print("🏥 Enriching Facilities with Real CMS Medicaid Data...")
    
    if not CMS_FILE.exists():
        print("❌ CMS File not found. Please download 'Hospital General Information.csv' from CMS.gov")
        print("   Place it in data/raw/ and rename to 'cms_hospitals.csv'")
        print("   ⚠️ Falling back to Random Simulation for now...")
        return False

    # 1. Read CMS Data
    # Logic: If it is in the CMS database and matches criteria, we assume it accepts public insurance.
    try:
        df_cms = pd.read_csv(CMS_FILE, encoding='latin1') 
        # Simplify: Keep only PA hospitals
        df_cms = df_cms[df_cms['State'] == 'PA']
        
        # Extract facility names
        cms_names = df_cms['Facility Name'].str.upper().tolist()
        
        print(f"   ✅ Loaded {len(cms_names)} PA hospitals from CMS database.")
        
    except Exception as e:
        print(f"   ❌ Error reading CMS CSV: {e}")
        return False

    # 2. Read Local Facilities Data
    gdf_fac = gpd.read_file(FACILITIES_FILE)
    
    # 3. Fuzzy Match
    # Match OpenDataPhilly names vs CMS names (e.g., "Hosp of Univ of PA" vs "HOSPITAL OF THE UPENN")
    
    matched_count = 0
    accepts_list = []
    
    for i, row in gdf_fac.iterrows():
        local_name = str(row['name']).upper()
        
        # Skip unknown names in buffer
        if "UNKNOWN" in local_name:
            accepts_list.append(False) 
            continue
            
        # Find closest match in CMS list
        matches = get_close_matches(local_name, cms_names, n=1, cutoff=0.6)
        
        if matches:
            # Found! It is a CMS certified hospital
            accepts_list.append(True)
            matched_count += 1
        else:
            # Not found. Might be private practice or specialty.
            # Exception: Public Health Centers should be True.
            if "HEALTH CENTER" in local_name:
                accepts_list.append(True) 
            else:
                accepts_list.append(False)
    
    gdf_fac['accepts_medicaid'] = accepts_list
    
    print(f"   ✅ Matched {matched_count} hospitals with CMS records.")
    print(f"   ✅ Health Centers manually set to True.")
    print(f"   📊 Total Medicaid-Accepting Facilities: {sum(accepts_list)} / {len(gdf_fac)}")
    
    # Save
    gdf_fac.to_file(FACILITIES_FILE, driver="GeoJSON")
    print("   💾 Updated facilities file saved.")
    return True

if __name__ == "__main__":
    enrich_facilities_with_cms()

🏥 Enriching Facilities with Real CMS Medicaid Data...
   ✅ Loaded 187 PA hospitals from CMS database.
   ✅ Matched 85 hospitals with CMS records.
   ✅ Health Centers manually set to True.
   📊 Total Medicaid-Accepting Facilities: 85 / 146
   💾 Updated facilities file saved.


In [23]:
import networkx as nx
BASE_DIR = Path(r".")
DATA_DIR = BASE_DIR / "data"
DATA_RAW = DATA_DIR / "raw"
DATA_PROCESSED = DATA_DIR / "processed"
RESULTS_DIR = BASE_DIR / "results"
RESULTS_DIR.mkdir(parents=True, exist_ok=True)
TARGET_CRS = "EPSG:32129"

# E2SFCA Parameters
GAUSSIAN_THRESHOLD = 30 
BETA = - (GAUSSIAN_THRESHOLD**2) / np.log(0.01)

def gaussian_weight(t_min):
    if t_min > GAUSSIAN_THRESHOLD: return 0
    return np.exp(- (t_min**2) / BETA)

def calculate_od_matrix(G, tracts, facilities):
    print("⏱️  Calculating OD Matrix...")
    tract_centroids = tracts.geometry.centroid
    facility_points = facilities.geometry
    
    # Snap points to network
    orig_nodes = ox.nearest_nodes(G, X=tract_centroids.x, Y=tract_centroids.y)
    dest_nodes = ox.nearest_nodes(G, X=facility_points.x, Y=facility_points.y)
    
    unique_origs = list(set(orig_nodes))
    od_times = {}
    
    # Calculate travel times
    for o_node in unique_origs:
        lengths = nx.single_source_dijkstra_path_length(
            G, o_node, weight='travel_time', cutoff=(GAUSSIAN_THRESHOLD+5)*60
        )
        od_times[o_node] = lengths
        
    return orig_nodes, dest_nodes, od_times

def run_e2sfca(tracts, facilities, orig_nodes, dest_nodes, od_times, subset_name="all"):
    print(f"   ...Calculating E2SFCA for subset: {subset_name}")
    
    # Determine population column
    pop_col = 'population' if 'population' in tracts.columns else 'E_TOTPOP'
    if pop_col not in tracts.columns: 
        tracts['dummy_pop'] = 1000
        pop_col = 'dummy_pop'
        
    # Determine beds column
    bed_col = 'BEDS'
    if bed_col not in facilities.columns: 
        facilities[bed_col] = 1.0

    # Step 1: Calculate R_j (Supply-to-Demand Ratio)
    r_j_list = []
    
    # Use enumerate to align with dest_nodes list index
    for idx, (original_idx, fac) in enumerate(facilities.iterrows()):
        f_node = dest_nodes[idx] 
        cap = float(fac[bed_col]) if pd.notnull(fac[bed_col]) else 1.0
        
        weighted_pop = 0
        for i, tract in tracts.iterrows():
            t_node = orig_nodes[i]
            if t_node in od_times and f_node in od_times[t_node]:
                t_min = od_times[t_node][f_node] / 60
                weighted_pop += tract[pop_col] * gaussian_weight(t_min)
        
        r_j_list.append(cap / weighted_pop if weighted_pop > 0 else 0)
    
    facilities[f'R_{subset_name}'] = r_j_list
    
    # Step 2: Calculate A_i (Accessibility Score)
    access_scores = []
    for i, tract in tracts.iterrows():
        t_node = orig_nodes[i]
        sum_r = 0
        for idx, (original_idx, fac) in enumerate(facilities.iterrows()):
            f_node = dest_nodes[idx]
            if t_node in od_times and f_node in od_times[t_node]:
                t_min = od_times[t_node][f_node] / 60
                sum_r += fac[f'R_{subset_name}'] * gaussian_weight(t_min)
        access_scores.append(sum_r * 1000)
        
    return access_scores

def recommend_sites(df_res, pop_col):
    print("📍 Calculating Optimal New Sites (Gap Analysis)...")
    
    # Social Vulnerability Index check
    svi_col = 'RPL_THEMES' if 'RPL_THEMES' in df_res.columns else 'SVI'
    if svi_col in df_res.columns:
        svi_factor = df_res[svi_col].replace(-999, 0.5).fillna(0.5)
    else:
        svi_factor = 0.5
        
    # Prevent division by zero
    access_safe = df_res['access_medicaid'].replace(0, 0.0001)
    
    # Gap Index = (Demand * Vulnerability) / Supply
    df_res['gap_index'] = (df_res[pop_col] * svi_factor) / access_safe
    
    top_3 = df_res.nlargest(3, 'gap_index')
    recs = top_3[['GEOID', 'gap_index', 'geometry', pop_col, 'access_medicaid']].copy()
    recs['geometry'] = recs.geometry.centroid
    recs['suggestion'] = ["Priority Site #1", "Priority Site #2", "Priority Site #3"]
    
    return recs

def main():
    print("🚀 STARTING FINAL ANALYSIS...")
    
    # Load Data
    tracts = gpd.read_file(DATA_PROCESSED / "tracts_pop_svi_projected.geojson")
    facilities = gpd.read_file(DATA_PROCESSED / "facilities_with_buffer.geojson")
    
    # Load Network
    graph_path = DATA_RAW / "philly_drive_network.graphml"
    G = ox.load_graphml(graph_path)
    G = ox.project_graph(G, to_crs=TARGET_CRS)
    G = ox.add_edge_speeds(G)
    G = ox.add_edge_travel_times(G)
    
    # Pre-calculate OD Matrix
    orig, dest, od = calculate_od_matrix(G, tracts, facilities)
    
    # -----------------------------------------------------
    print("🧮 Phase 1: General Accessibility (All Facilities)")
    scores_all = run_e2sfca(tracts, facilities, orig, dest, od, "all")
    tracts['access_all'] = scores_all
    
    # -----------------------------------------------------
    print("🧮 Phase 2: Equity Accessibility (Medicaid Only)")
    if 'accepts_medicaid' in facilities.columns:
        # [CRITICAL FIX] Must reset index so 'iterrows' aligns with dest_nodes list index
        medicaid_facs = facilities[facilities['accepts_medicaid'] == True].copy().reset_index(drop=True)
        
        # Re-calculate destination nodes for this specific subset
        dest_nodes_med = ox.nearest_nodes(G, X=medicaid_facs.geometry.x, Y=medicaid_facs.geometry.y)
        
        scores_med = run_e2sfca(tracts, medicaid_facs, orig, dest_nodes_med, od, "medicaid")
        tracts['access_medicaid'] = scores_med
    else:
        print("⚠️ 'accepts_medicaid' column missing. Skipping Phase 2.")
        tracts['access_medicaid'] = scores_all 
    
    # -----------------------------------------------------
    # Site Recommendations
    pop_col = 'population' if 'population' in tracts.columns else 'E_TOTPOP'
    if pop_col not in tracts.columns: pop_col = 'dummy_pop'
    
    recs_gdf = recommend_sites(tracts, pop_col)
    
    # Output Files
    out_res = RESULTS_DIR / "final_E2SFCA_results.geojson"
    tracts.to_file(out_res, driver="GeoJSON")
    
    out_rec = RESULTS_DIR / "recommended_new_sites.geojson"
    recs_gdf.to_file(out_rec, driver="GeoJSON")
    
    print(f"\n✅ Analysis Complete!")
    print(f"   - Main Results: {out_res}")
    print(f"   - Recommendations: {out_rec}")

if __name__ == "__main__":
    main()

🚀 STARTING FINAL ANALYSIS...
⏱️  Calculating OD Matrix...
🧮 Phase 1: General Accessibility (All Facilities)
   ...Calculating E2SFCA for subset: all
🧮 Phase 2: Equity Accessibility (Medicaid Only)
   ...Calculating E2SFCA for subset: medicaid
📍 Calculating Optimal New Sites (Gap Analysis)...

✅ Analysis Complete!
   - Main Results: results\final_E2SFCA_results.geojson
   - Recommendations: results\recommended_new_sites.geojson


In [24]:
import osmnx as ox
import geopandas as gpd
import pandas as pd
from pathlib import Path
import warnings

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

# ---------------------------------------------------------
# 1. Project Configuration & Paths
# ---------------------------------------------------------
# Base directory of your project
BASE_DIR = Path(".")

# Data directories
DATA_DIR = BASE_DIR / "data"
DATA_RAW = DATA_DIR / "raw"
DATA_PROCESSED = DATA_DIR / "processed"

# Ensure processed directory exists
DATA_PROCESSED.mkdir(parents=True, exist_ok=True)

# Target Coordinate Reference System (UTM Zone 18N for PA/NJ - Meters)
# We use this for accurate centroid calculation and distance measurement
TARGET_CRS = "EPSG:32129"  # Or EPSG:32618

def fetch_buffer_hospitals_robust():
    """
    Robust Version: Fetches hospitals from the surrounding region (PA Suburbs + NJ)
    using a Bounding Box to avoid timeouts and edge effects.
    """
    print("🏥 Fetching Buffer Hospitals (Using Bounding Box Strategy)...")
    
    # ---------------------------------------------------------
    # 2. Define Bounding Box (Philly + NJ + PA Suburbs)
    # ---------------------------------------------------------
    # Coordinates cover:
    # North: Montgomery/Bucks | South: Gloucester (NJ)
    # West: Chester/Delaware  | East: Burlington/Camden (NJ)
    north, south = 40.40, 39.60
    east, west = -74.70, -75.60
    
    tags = {'amenity': 'hospital'}
    
    try:
        print(f"   ...Querying OSM within box (N:{north}, S:{south}, E:{east}, W:{west})...")
        
        gdf_buffer = None

        # --- OSMnx Version Compatibility Check ---
        if hasattr(ox, 'features_from_bbox'):
            try:
                gdf_buffer = ox.features_from_bbox(bbox=(north, south, east, west), tags=tags)
            except TypeError:
                gdf_buffer = ox.features_from_bbox(north, south, east, west, tags)
        elif hasattr(ox, 'geometries_from_bbox'):
            gdf_buffer = ox.geometries_from_bbox(north, south, east, west, tags)
        else:
            try:
                gdf_buffer = ox.features.features_from_bbox(bbox=(north, south, east, west), tags=tags)
            except:
                pass

        if gdf_buffer is None or len(gdf_buffer) == 0:
             raise ValueError("No data returned from OSMnx.")

        # ---------------------------------------------------------
        # 3. Data Cleaning & Warning Fix
        # ---------------------------------------------------------
        print(f"   ...Downloaded {len(gdf_buffer)} raw features. Cleaning...")

        # [CRITICAL FIX] 
        # Project to Meters (Target CRS) BEFORE calculating centroid.
        # This fixes the "Geometry is in a geographic CRS" warning.
        gdf_buffer = gdf_buffer.to_crs(TARGET_CRS)
        
        # Now it is safe to calculate centroids
        gdf_buffer['geometry'] = gdf_buffer.geometry.centroid
        
        # Reset index
        gdf_buffer = gdf_buffer.reset_index(drop=True)
        
        # Ensure 'name' column exists
        if 'name' not in gdf_buffer.columns:
            gdf_buffer['name'] = 'Unknown Buffer Hospital'
            
        # Keep only necessary columns
        gdf_buffer = gdf_buffer[['name', 'geometry']].copy()
        gdf_buffer['facility_type'] = 'Buffer Hospital'
        
        # Assign default bed count
        gdf_buffer['BEDS'] = 150 
        
        # ---------------------------------------------------------
        # 4. Merge with Existing Philly Facilities
        # ---------------------------------------------------------
        philly_path = DATA_PROCESSED / "facilities_projected.geojson"
        
        if philly_path.exists():
            philly_facilities = gpd.read_file(philly_path)
            
            # Ensure Philly data is in the same CRS
            if philly_facilities.crs != TARGET_CRS:
                philly_facilities = philly_facilities.to_crs(TARGET_CRS)

            # Concatenate
            combined_facilities = pd.concat([philly_facilities, gdf_buffer], ignore_index=True)
            
            # Deduplicate (Remove hospitals that are practically in the same spot)
            # Round coordinates to nearest 1 meter to find duplicates
            combined_facilities['x_round'] = combined_facilities.geometry.x.round(0)
            combined_facilities['y_round'] = combined_facilities.geometry.y.round(0)
            
            combined_facilities = combined_facilities.drop_duplicates(subset=['x_round', 'y_round'])
            combined_facilities = combined_facilities.drop(columns=['x_round', 'y_round'])
            
            # Save final file using the defined path constant
            out_path = DATA_PROCESSED / "facilities_with_buffer.geojson"
            combined_facilities.to_file(out_path, driver="GeoJSON")
            
            print(f"   ✅ Success! Total Facilities: {len(combined_facilities)}")
            print(f"      - Original Philly: {len(philly_facilities)}")
            print(f"      - Buffer Raw: {len(gdf_buffer)}")
            print(f"      - After Deduplication: {len(combined_facilities)}")
            print(f"   💾 Saved to: {out_path}")
        else:
            print(f"   ❌ Error: Philly facilities file not found at {philly_path}")
            print("      Please run the previous processing step first.")
        
    except Exception as e:
        print(f"   ❌ Failed to fetch buffer hospitals: {e}")

if __name__ == "__main__":
    fetch_buffer_hospitals_robust()

🏥 Fetching Buffer Hospitals (Using Bounding Box Strategy)...
   ...Querying OSM within box (N:40.4, S:39.6, E:-74.7, W:-75.6)...
   ...Downloaded 110 raw features. Cleaning...
   ✅ Success! Total Facilities: 146
      - Original Philly: 36
      - Buffer Raw: 110
      - After Deduplication: 146
   💾 Saved to: data\processed\facilities_with_buffer.geojson


In [25]:
def create_publication_map(gdf, column, title, output_path, cmap='viridis', scheme='FisherJenks', k=5, kind='area', add_basemap=True):
    """
    Generates a high-quality, scientifically rigorous map.
    Handles projection, basemaps, scale bars, and legends automatically.
    """
    # 1. Reproject to Web Mercator for Basemap compatibility
    gdf_plot = gdf.to_crs(CRS_VISUAL)
    
    # 2. Setup Figure
    fig, ax = plt.subplots(figsize=(12, 12))
    
    # 3. Plot Data based on Type
    if kind == 'area':
        # Handle NaN values to prevent plotting errors
        if column in gdf_plot.columns:
            gdf_plot = gdf_plot.dropna(subset=[column])
            
        gdf_plot.plot(
            column=column,
            ax=ax,
            cmap=cmap,
            scheme=scheme,
            k=k,
            alpha=0.7,
            edgecolor='white',
            linewidth=0.2,
            legend=True,
            legend_kwds={
                'loc': 'lower right',
                'title': "Legend",
                'frameon': True,
                'fmt': '{:.2f}', # Force 2 decimal places (e.g., 0.55 instead of 0.55432)
                'bbox_to_anchor': (1, 0.25)
            }
        )
    elif kind == 'point':
        gdf_plot.plot(
            ax=ax,
            color='#d62728', # Scientific Red
            markersize=200,
            edgecolor='white',
            linewidth=1.5,
            marker='*',
            zorder=10,
            label='Proposed Locations'
        )
        # Manually add legend for points
        ax.legend(loc='lower right', fontsize=12)

    # 4. Add Basemap (CartoDB Positron - Clean and Scientific)
    if add_basemap:
        try:
            ctx.add_basemap(ax, source=ctx.providers.CartoDB.Positron)
        except Exception as e:
            print(f"⚠️ Basemap warning: {e}")

    # 5. Scientific Annotations
    # Title
    ax.set_title(title, fontsize=16, fontweight='bold', pad=20)
    
    # North Arrow (Simulated)
    x, y, arrow_length = 0.05, 0.95, 0.1
    ax.annotate('N', xy=(x, y), xytext=(x, y-arrow_length),
                arrowprops=dict(facecolor='black', width=5, headwidth=15),
                ha='center', va='center', fontsize=20, xycoords=ax.transAxes)

    # Scale Bar (Approximation based on CRS units)
    # Since we projected to 3857 (Meters), we can use matplotlib_scalebar
    scalebar = ScaleBar(1, "m", location="lower left", box_alpha=0.5)
    ax.add_artist(scalebar)

    # Source Credit
    plt.figtext(0.5, 0.05, "Data Sources: OpenStreetMap, CDC SVI 2020, CMS Hospital Data | Projection: EPSG:32129", 
                ha='center', fontsize=8, color='#555555')

    # 6. Clean Axis
    ax.set_axis_off()
    
    # 7. Save
    plt.savefig(output_path, bbox_inches='tight', dpi=300)
    plt.close() # Close plot to free memory
    print(f"🖼️ Saved: {output_path.name}")

In [27]:
def module_1_demonstrate_error():
    print("🚀 Module 1: Visualizing the Edge Effect Bias...")
    
    # Load Data
    tracts_path = DATA_PROCESSED / "tracts_pop_svi_projected.geojson"
    fac_path = DATA_PROCESSED / "facilities_with_buffer.geojson"
    
    if not fac_path.exists():
        print("❌ Data missing. Please run previous data processing steps.")
        return

    # Load and ensure Metric CRS for accurate spatial calculations
    tracts = gpd.read_file(tracts_path).to_crs(CRS_METRIC)
    facilities = gpd.read_file(fac_path).to_crs(CRS_METRIC)
    
    # --- FIX: SPATIAL FILTERING ---
    # Instead of relying on the 'facility_type' column (which may have errors),
    # we determine the status based on actual location geometry.
    
    # 1. Create a unified polygon for the Philadelphia boundary
    philly_boundary = tracts.unary_union
    
    # 2. Check which facilities are strictly INSIDE the boundary
    # This returns a boolean (True/False) series
    is_inside_philly = facilities.geometry.within(philly_boundary)
    
    # 3. Split the data based on geometry
    philly_only_facs = facilities[is_inside_philly]  # Strictly Inside
    buffer_facs = facilities[~is_inside_philly]      # Strictly Outside (The Inverse)
    
    # --- VISUALIZATION ---
    output_path = DIRS["error"] / "01_error_edge_effect_demonstration_fixed.png"
    
    fig, ax = plt.subplots(figsize=(10, 10))
    
    # Plot Tracts (The Grey City Area)
    tracts.to_crs(CRS_VISUAL).plot(ax=ax, color='#f0f0f0', edgecolor='#cccccc')
    
    # Plot Philadelphia Facilities (Blue Dots) - strictly inside
    philly_only_facs.to_crs(CRS_VISUAL).plot(
        ax=ax, 
        color='blue', 
        markersize=30, 
        label='Philadelphia Hospitals (Included)', 
        alpha=0.7
    )
    
    # Plot Buffer Facilities (Red X) - strictly outside
    buffer_facs.to_crs(CRS_VISUAL).plot(
        ax=ax, 
        color='red', 
        marker='x', 
        markersize=50, 
        label='IGNORED Buffer Hospitals', 
        alpha=0.9
    )
    
    # Add Basemap for context
    try:
        ctx.add_basemap(ax, source=ctx.providers.CartoDB.Positron)
    except Exception as e:
        print(f"⚠️ Could not load basemap: {e}")
    
    ax.set_title(
        "Methodological Flaw: The Edge Effect\n(Red 'X' marks are facilities outside the city boundary)", 
        fontsize=14, 
        color='darkred'
    )
    
    ax.legend(loc='lower right')
    ax.set_axis_off()
    
    plt.savefig(output_path, bbox_inches='tight', dpi=300)
    plt.close()
    print(f"✅ Corrected Edge Effect map generated at: {output_path}")

module_1_demonstrate_error()

🚀 Module 1: Visualizing the Edge Effect Bias...


C:\Users\14175\AppData\Local\Temp\ipykernel_6864\2111117143.py:21: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  philly_boundary = tracts.unary_union


✅ Corrected Edge Effect map generated at: results\01_missteps_edge_effects\01_error_edge_effect_demonstration_fixed.png


In [28]:
def module_2_visualize_process():
    print("🚀 Module 2: Generating Intermediate Process Maps...")
    
    tract_path = DATA_PROCESSED / "tracts_pop_svi_projected.geojson"
    gdf = gpd.read_file(tract_path).to_crs(CRS_METRIC)
    
    # Clean SVI Data
    if 'RPL_THEMES' in gdf.columns:
        gdf['SVI_Score'] = gdf['RPL_THEMES'].replace(-999, np.nan)
    else:
        print("⚠️ SVI column missing, creating dummy for demo.")
        gdf['SVI_Score'] = np.random.rand(len(gdf))
        
    # Map 1: Social Vulnerability Index
    create_publication_map(
        gdf, 
        column='SVI_Score', 
        title="Input Variable: Social Vulnerability Index (SVI)\nPhiladelphia (2020)", 
        output_path=DIRS["process"] / "02_process_SVI_distribution.png",
        cmap='OrRd', # Orange-Red for danger/vulnerability
        scheme='Quantiles'
    )
    
    # Map 2: Population Density (Calculated in Meters)
    # Area is in Sq Meters. Convert to Sq Km -> Area / 10^6
    gdf['pop_density_sqkm'] = gdf['population'] / (gdf.geometry.area / 1_000_000)
    
    create_publication_map(
        gdf,
        column='pop_density_sqkm',
        title="Input Variable: Population Density\n(People per Sq. Km)",
        output_path=DIRS["process"] / "02_process_pop_density.png",
        cmap='Blues',
        scheme='FisherJenks' # Good for density with outliers
    )

module_2_visualize_process()

🚀 Module 2: Generating Intermediate Process Maps...
🖼️ Saved: 02_process_SVI_distribution.png
🖼️ Saved: 02_process_pop_density.png


In [29]:
import folium
# ---------------------------------------------------------
# OPTIMIZED MODULE 3: FINAL VISUALIZATION SUITE (TRANSPARENT NODATA)
# ---------------------------------------------------------
def module_3_complete_suite():
    print("🚀 Module 3: Generating All Final Deliverables (Fixed Scale & Transparency)...")
    
    # Define Paths
    res_path = BASE_DIR / "results" / "final_E2SFCA_results.geojson"
    rec_path = BASE_DIR / "results" / "recommended_new_sites.geojson"
    
    if not res_path.exists():
        print("❌ Data not found. Please run the analysis module first.")
        return

    # Load Data
    gdf_res = gpd.read_file(res_path)
    if rec_path.exists():
        gdf_rec = gpd.read_file(rec_path)
    else:
        gdf_rec = None

    # =========================================================
    # PART 1: STATIC EQUITY GAP MAP (Refined PNG)
    # =========================================================
    print("   ... 1/2 Generating Static Equity Gap Map (PNG)")
    
    gdf_static = gdf_res.to_crs(CRS_METRIC).copy()
    
    # Calculate Gap Percentage
    access_all_safe = gdf_static['access_all'].replace(0, 0.0001)
    gdf_static['equity_gap_pct'] = ((gdf_static['access_all'] - gdf_static['access_medicaid']) / access_all_safe) * 100
    gdf_static['equity_gap_pct'] = gdf_static['equity_gap_pct'].clip(0, 100).fillna(0)
    
    # Reproject
    gdf_plot = gdf_static.to_crs(CRS_VISUAL)
    
    # Check if data is too uniform
    scheme = 'EqualInterval' if gdf_static['equity_gap_pct'].nunique() <= 1 else 'Quantiles'

    fig, ax = plt.subplots(figsize=(12, 12))
    
    gdf_plot.plot(
        column='equity_gap_pct',
        ax=ax,
        cmap='OrRd', 
        scheme=scheme,
        k=5,
        alpha=0.85,
        edgecolor='black',
        linewidth=0.1,
        legend=True,
        legend_kwds={
            'loc': 'upper right',
            'title': 'Access Loss (%)',
            'framealpha': 0.9,
            'facecolor': 'white'
        }
    )

    # Clean Legend
    leg = ax.get_legend()
    if leg:
        for text in leg.get_texts():
            try:
                parts = text.get_text().replace('(', '').replace(']', '').replace('[', '').split(',')
                if len(parts) == 2:
                    text.set_text(f"{float(parts[0]):.0f}% - {float(parts[1]):.0f}%")
            except: pass 

    try: ctx.add_basemap(ax, source=ctx.providers.CartoDB.Positron)
    except: pass
    
    ax.set_axis_off()
    ax.set_title("The Medicaid Equity Gap\n(% Access Lost Due to Insurance Barriers)", fontsize=16, fontweight='bold', pad=20)
    
    out_png = DIRS["final"] / "03_final_equity_gap_map_optimized.png"
    plt.savefig(out_png, bbox_inches='tight', dpi=300)
    plt.close()
    print(f"      ✅ Saved: {out_png.name}")

    # =========================================================
    # PART 2: INTERACTIVE RECOMMENDATION MAP (HTML) - TRANSPARENT NODATA
    # =========================================================
    print("   ... 2/2 Generating Interactive Map (HTML)")
    
    if gdf_rec is None:
        return

    # Reproject
    gdf_res_web = gdf_res.to_crs("EPSG:4326")
    gdf_rec_web = gdf_rec.to_crs("EPSG:4326")

    # Handle SVI Column
    svi_col = 'SVI' if 'SVI' in gdf_res_web.columns else 'RPL_THEMES'
    if svi_col not in gdf_res_web.columns:
        gdf_res_web[svi_col] = 0.5 
    
    # --- KEY FIX: DATA CLEANING & TRANSPARENCY ---
    # 1. Replace -999 with NaN so they are treated as missing
    gdf_res_web[svi_col] = gdf_res_web[svi_col].replace(-999, np.nan)
    
    # Initialize Map
    m = folium.Map(location=[39.9526, -75.1652], zoom_start=11, tiles="CartoDB positron")

    # Define bins to force valid range 0-1
    bins = [0, 0.2, 0.4, 0.6, 0.8, 1.0]

    # Add SVI Choropleth (Background)
    folium.Choropleth(
        geo_data=gdf_res_web,
        name="Social Vulnerability (SVI)",
        data=gdf_res_web,
        columns=['GEOID', svi_col],
        key_on='feature.properties.GEOID',
        fill_color='Purples',
        fill_opacity=0.7,
        line_opacity=0.1,
        # --- TRANSPARENCY FIX ---
        nan_fill_opacity=0,    # Make missing values (NaN/-999) fully transparent
        nan_fill_color='white', # Fallback (won't be seen if opacity is 0)
        # ------------------------
        legend_name='Social Vulnerability (0=Low, 1=High)',
        bins=bins,
        highlight=True
    ).add_to(m)

    # Add Tooltips
    style_function = lambda x: {'fillColor': '#ffffff', 'color':'#000000', 'fillOpacity': 0.1, 'weight': 0.1}
    highlight_function = lambda x: {'fillColor': '#000000', 'color':'#000000', 'fillOpacity': 0.50, 'weight': 0.1}
    
    hover_layer = folium.features.GeoJson(
        gdf_res_web,
        style_function=style_function,
        control=False,
        highlight_function=highlight_function,
        tooltip=folium.features.GeoJsonTooltip(
            fields=[svi_col, 'access_all'],
            aliases=['SVI Score:', 'Access Score:'],
            style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;")
        )
    )
    m.add_child(hover_layer)

    # Add Recommended Sites
    for _, row in gdf_rec_web.iterrows():
        folium.Marker(
            location=[row.geometry.y, row.geometry.x],
            popup=folium.Popup(f"<b>{row['suggestion']}</b><br>Optimized for High Equity Gain", max_width=300),
            tooltip=row['suggestion'],
            icon=folium.Icon(color='red', icon='star', prefix='fa')
        ).add_to(m)

    folium.LayerControl().add_to(m)
    
    out_html = DIRS["final"] / "04_final_recommendation_map_interactive.html"
    m.save(out_html)
    print(f"      ✅ Saved: {out_html.name}")
    print("      👉 Open this file to see transparent -999 areas.")

# Run module 3
module_3_complete_suite()

🚀 Module 3: Generating All Final Deliverables (Fixed Scale & Transparency)...
   ... 1/2 Generating Static Equity Gap Map (PNG)
      ✅ Saved: 03_final_equity_gap_map_optimized.png
   ... 2/2 Generating Interactive Map (HTML)
      ✅ Saved: 04_final_recommendation_map_interactive.html
      👉 Open this file to see transparent -999 areas.


In [30]:
BASE_DIR = Path(r".")
RESULTS_DIR = BASE_DIR / "results"

# FIX: Simplified path to avoid "results/results" duplication
HTML_OUT = RESULTS_DIR / "03_final_deliverables" / "index.html"
def generate_web_dashboard():
    print("🚀 Generating Optimized Interactive Dashboard...")
    
    res_path = RESULTS_DIR / "final_E2SFCA_results.geojson"
    if not res_path.exists():
        print("❌ Results file not found.")
        return

    gdf = gpd.read_file(res_path).to_crs("EPSG:4326")
    
    # 1. Clean SVI Data (Replace -999 with NaN for transparency)
    if 'RPL_THEMES' in gdf.columns:
        gdf['SVI'] = gdf['RPL_THEMES'].replace(-999, np.nan)
    else:
        gdf['SVI'] = np.nan

    # 2. Calculate Equity Gap (% Access Lost)
    # Logic: (Access All - Access Medicaid) / Access All
    gdf['Equity_Gap'] = (
        ((gdf['access_all'] - gdf['access_medicaid']) / gdf['access_all'].replace(0, 0.0001)) * 100
    ).clip(0, 100).fillna(0)

    # 3. Calculate "Weighted Impact" (New Metric)
    # Logic: Equity Gap * SVI. 
    # Highlights areas that lose access AND are socially vulnerable.
    gdf['Weighted_Impact'] = gdf['Equity_Gap'] * gdf['SVI'].fillna(0)

    gdf['Access_Score'] = gdf['access_medicaid']

    # ---------------- DYNAMIC SCALING LOGIC ----------------
    # Calculate the 95th percentile to ignore extreme outliers.
    # This ensures the map colors aren't washed out by one tiny area with 100% loss.
    gap_max = gdf['Equity_Gap'].quantile(0.95)
    impact_max = gdf['Weighted_Impact'].quantile(0.95)
    
    # Ensure min threshold to prevent crashes if all data is 0
    if gap_max < 1: gap_max = 1.0 
    if impact_max < 0.1: impact_max = 0.1

    print(f"📊 Dynamic Scale Max - Equity Gap: {gap_max:.2f}%")
    print(f"📊 Dynamic Scale Max - Weighted Impact: {impact_max:.2f}")

    # ---------------- VISUALIZATION ----------------
    def get_map(metric):
        # Dynamic Configuration Dictionary
        configs = {
            'SVI': {
                'cmap': 'OrRd', 
                'title': 'Social Vulnerability (SVI)', 
                'clim': (0, 1)
            },
            'Equity_Gap': {
                'cmap': 'Plasma', 
                'title': f'Equity Gap (% Access Lost) [Scale: 0 - {gap_max:.1f}%]', 
                'clim': (0, gap_max) # <--- Dynamic Scale used here
            },
            'Weighted_Impact': {
                'cmap': 'Inferno', 
                'title': 'Priority Index (Gap x SVI)', 
                'clim': (0, impact_max)
            },
            'Access_Score': {
                'cmap': 'Viridis', 
                'title': 'Raw Accessibility Score', 
                'clim': None # Auto-range
            }
        }
        
        c = configs[metric]
        clim_args = {}
        if c['clim']:
            clim_args['clim'] = c['clim']

        return gdf.hvplot.polygons(
            c=metric,
            cmap=c['cmap'],
            title=c['title'],
            geo=True,
            tiles='CartoLight',
            alpha=0.75,
            hover_cols=['GEOID', 'SVI', 'Equity_Gap', 'Weighted_Impact', 'Access_Score'],
            line_width=0.05,
            line_color='white',
            frame_height=600,
            aspect='equal',
            missing_color='rgba(0,0,0,0)', 
            **clim_args
        )

    # Widgets
    metrics_list = ['Equity_Gap', 'Weighted_Impact', 'SVI', 'Access_Score']
    select_widget = pn.widgets.Select(name='Select Metric:', options=metrics_list)
    
    desc = pn.pane.Markdown("""
    # 🏥 Philadelphia Healthcare Equity Explorer
    **Thesis Project: Beyond Physical Distance**
    
    * **Equity Gap:** Percentage of access lost when relying solely on Medicaid facilities. (Scaled dynamically).
    * **Priority Index:** Combines the Equity Gap with SVI. High values = Vulnerable areas losing significant access.
    * **SVI:** Social Vulnerability Index.
    """)
    
    dashboard = pn.Column(
        desc,
        select_widget,
        pn.bind(get_map, metric=select_widget)
    )
    
    dashboard.save(str(HTML_OUT), embed=True)
    print(f"✅ Dashboard saved to: {HTML_OUT}")
    return dashboard

if __name__ == "__main__":
    generate_web_dashboard()

🚀 Generating Optimized Interactive Dashboard...
📊 Dynamic Scale Max - Equity Gap: 30.28%
📊 Dynamic Scale Max - Weighted Impact: 25.58


  0%|                                                                                            | 0/4 [00:00<?, ?it/s]

 25%|█████████████████████                                                               | 1/4 [00:01<00:03,  1.19s/it]

 50%|██████████████████████████████████████████                                          | 2/4 [00:02<00:02,  1.19s/it]

 75%|███████████████████████████████████████████████████████████████                     | 3/4 [00:03<00:01,  1.20s/it]

✅ Dashboard saved to: results\03_final_deliverables\index.html


In [31]:
# ---------------- CONFIGURATION ----------------
BASE_DIR = Path(".")
DATA_PROCESSED = BASE_DIR / "data" / "processed"
DATA_RAW = BASE_DIR / "data" / "raw"
RESULTS_DIR = BASE_DIR / "results/sensitivity/"
RESULTS_DIR.mkdir(parents=True, exist_ok=True)
TARGET_CRS = "EPSG:32129"

def gaussian_weight(t, t_0):
    """Calculates Gaussian weight based on dynamic threshold t_0."""
    if t > t_0: return 0
    beta = - (t_0 ** 2) / np.log(0.01)
    return np.exp(- (t ** 2) / beta)

def run_sensitivity_analysis():
    print("🚀 Starting Sensitivity Analysis (15, 20, 30 mins)...")

    # 1. Load Data
    tracts = gpd.read_file(DATA_PROCESSED / "tracts_pop_svi_projected.geojson")
    facilities = gpd.read_file(DATA_PROCESSED / "facilities_with_buffer.geojson")
    
    # Load Network
    print("   ...Loading road network")
    G = ox.load_graphml(DATA_RAW / "philly_drive_network.graphml")
    G = ox.project_graph(G, to_crs=TARGET_CRS)
    G = ox.add_edge_speeds(G)
    G = ox.add_edge_travel_times(G)

    # 2. Calculate OD Matrix (Max cutoff 45 mins to cover all scenarios)
    print("   ...Calculating OD Matrix")
    tract_centroids = tracts.geometry.centroid
    facility_points = facilities.geometry
    
    orig_nodes = ox.nearest_nodes(G, X=tract_centroids.x, Y=tract_centroids.y)
    dest_nodes = ox.nearest_nodes(G, X=facility_points.x, Y=facility_points.y)
    
    unique_origs = list(set(orig_nodes))
    od_times = {}
    
    for o_node in unique_origs:
        lengths = nx.single_source_dijkstra_path_length(
            G, o_node, weight='travel_time', cutoff=45*60
        )
        od_times[o_node] = lengths

    # 3. Loop through thresholds
    thresholds = [15, 20, 30]
    
    # Determine columns
    pop_col = 'population' if 'population' in tracts.columns else 'E_TOTPOP'
    if pop_col not in tracts.columns: tracts['dummy_pop'] = 1000; pop_col = 'dummy_pop'
    
    bed_col = 'BEDS'
    if bed_col not in facilities.columns: facilities[bed_col] = 1.0

    for t_threshold in thresholds:
        print(f"   ...Processing {t_threshold} minute threshold")
        
        # --- Step 1: Supply-to-Demand Ratio (R_j) ---
        r_j_list = []
        for idx, row in facilities.iterrows():
            f_node = dest_nodes[idx]
            cap = float(row[bed_col])
            
            weighted_pop = 0
            for i, tract in tracts.iterrows():
                t_node = orig_nodes[i]
                if t_node in od_times and f_node in od_times[t_node]:
                    time_min = od_times[t_node][f_node] / 60
                    weighted_pop += tract[pop_col] * gaussian_weight(time_min, t_threshold)
            
            r_j_list.append(cap / weighted_pop if weighted_pop > 0 else 0)
        
        # --- Step 2: Accessibility Index (A_i) ---
        ai_list = []
        for i, tract in tracts.iterrows():
            t_node = orig_nodes[i]
            sum_r = 0
            for idx, row in facilities.iterrows():
                f_node = dest_nodes[idx]
                if t_node in od_times and f_node in od_times[t_node]:
                    time_min = od_times[t_node][f_node] / 60
                    # Use R_j calculated above
                    sum_r += r_j_list[idx] * gaussian_weight(time_min, t_threshold)
            ai_list.append(sum_r * 1000)
            
        tracts[f'access_{t_threshold}min'] = ai_list

    # 4. Identify Medical Deserts (Binary Definition)
    # Definition: Access score in bottom 20% AND Time to nearest > 30 mins
    print("   ...Identifying Medical Deserts")
    
    # Calculate nearest time
    min_times = []
    for i, tract in tracts.iterrows():
        t_node = orig_nodes[i]
        min_t = 9999
        for idx, row in facilities.iterrows():
            f_node = dest_nodes[idx]
            if t_node in od_times and f_node in od_times[t_node]:
                time_min = od_times[t_node][f_node] / 60
                if time_min < min_t: min_t = time_min
        min_times.append(min_t)
    
    tracts['nearest_time'] = min_times
    
    # Logic
    score_cutoff = tracts['access_30min'].quantile(0.20)
    tracts['is_desert'] = (tracts['access_30min'] < score_cutoff) | (tracts['nearest_time'] > 30)
    
    # 5. Save
    out_path = RESULTS_DIR / "sensitivity_analysis.geojson"
    tracts.to_file(out_path, driver="GeoJSON")
    print(f"✅ Sensitivity results saved to: {out_path}")

if __name__ == "__main__":
    run_sensitivity_analysis()

🚀 Starting Sensitivity Analysis (15, 20, 30 mins)...
   ...Loading road network
   ...Calculating OD Matrix
   ...Processing 15 minute threshold
   ...Processing 20 minute threshold
   ...Processing 30 minute threshold
   ...Identifying Medical Deserts
✅ Sensitivity results saved to: results\sensitivity\sensitivity_analysis.geojson


In [32]:
import geopandas as gpd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from pathlib import Path

# ---------------- CONFIGURATION ----------------
BASE_DIR = Path(".")
INPUT_FILE = BASE_DIR / "results" / "final_E2SFCA_results.geojson"
OUTPUT_DIR = BASE_DIR / "results/cluster"
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# Set plot style
sns.set_theme(style="whitegrid")
plt.rcParams['figure.dpi'] = 300

def get_cluster_label(row, stats):
    """
    Helper function to generate descriptive English labels 
    based on the cluster's average statistics.
    """
    cid = row['Cluster_ID']
    avg_svi = stats.loc[cid, 'SVI_Clean']
    avg_access = stats.loc[cid, 'access_all']
    
    # Define thresholds (you can adjust these based on your specific data distribution)
    # Using the global mean of the cluster centers to decide High/Low
    svi_threshold = stats['SVI_Clean'].mean()
    access_threshold = stats['access_all'].mean()

    svi_label = "High Vuln" if avg_svi > svi_threshold else "Low Vuln"
    
    # Creating descriptive Groups
    if avg_svi < 0.6: # Based on your plot, the split is clearly around 0.6
        return "Privileged (Low SVI, High Access)"
    else:
        # For High SVI, we have 3 levels of access
        if avg_access > 10.5:
             return "Supported (High SVI, High Access)"
        elif avg_access < 9.0:
             return "Critical Desert (High SVI, Low Access)"
        else:
             return "Moderate Access (High SVI)"

def run_clustering_and_stats():
    print("🤖 Running K-Means Clustering & Statistical Reporting...")
    
    if not INPUT_FILE.exists():
        print("❌ Input file not found. Run your main analysis first.")
        return

    gdf = gpd.read_file(INPUT_FILE)
    
    # 1. Prepare Data
    svi_col = 'RPL_THEMES' if 'RPL_THEMES' in gdf.columns else 'SVI'
    if svi_col in gdf.columns:
        gdf['SVI_Clean'] = gdf[svi_col].replace(-999, np.nan)
        gdf['SVI_Clean'] = gdf['SVI_Clean'].fillna(gdf['SVI_Clean'].mean())
    else:
        gdf['SVI_Clean'] = 0.5
        
    gdf_clean = gdf.dropna(subset=['access_all', 'SVI_Clean'])
    
    # 2. K-Means Clustering
    features = gdf_clean[['access_all', 'SVI_Clean']].copy()
    scaler = StandardScaler()
    features_scaled = scaler.fit_transform(features)
    
    kmeans = KMeans(n_clusters=4, random_state=42, n_init=10)
    gdf_clean['Cluster_ID'] = kmeans.fit_predict(features_scaled)
    
    # --- NEW STEP: ASSIGN ENGLISH NAMES ---
    # Calculate stats per cluster to determine what they represent
    cluster_stats = gdf_clean.groupby('Cluster_ID')[['access_all', 'SVI_Clean']].mean()
    
    # Apply the naming function
    gdf_clean['Cluster_Label'] = gdf_clean.apply(
        lambda row: get_cluster_label(row, cluster_stats), axis=1
    )
    
    # Save clustered data
    cluster_out = OUTPUT_DIR / "tracts_clustered.geojson"
    gdf_clean.to_file(cluster_out, driver="GeoJSON")
    print(f"    ✅ Clustered data saved to: {cluster_out}")

    # 3. Generate Summary CSV
    summary = gdf_clean.groupby('Cluster_Label')[['access_all', 'SVI_Clean']].mean()
    summary_path = OUTPUT_DIR / "cluster_summary.csv"
    summary.to_csv(summary_path)
      
    # Define a custom color palette logic if needed, or stick to viridis
    
    # A. Scatter Plot
    plt.figure(figsize=(12, 7)) # Made it slightly wider for the legend
    sns.scatterplot(
        data=gdf_clean, 
        x='SVI_Clean', 
        y='access_all', 
        hue='Cluster_Label', # <--- USE THE LABEL COLUMN
        palette='viridis',
        s=60, alpha=0.8
    )
    plt.title("Cluster Analysis: Accessibility vs. Social Vulnerability")
    plt.xlabel("Social Vulnerability (SVI)")
    plt.ylabel("Accessibility Score (E2SFCA)")
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', title='Population Group') # Move legend outside
    plt.tight_layout()
    plt.savefig(OUTPUT_DIR / "scatter_clusters_labeled.png")
    plt.close()
    
    # B. Box Plot
    plt.figure(figsize=(12, 7))
    sns.boxplot(
        x='Cluster_Label', # <--- USE THE LABEL COLUMN
        y='access_all', 
        data=gdf_clean, 
        palette='Set2'
    )
    plt.title("Accessibility Distribution by Population Group")
    plt.xlabel("Population Group")
    plt.xticks(rotation=15) # Rotate text so it fits
    plt.tight_layout()
    plt.savefig(OUTPUT_DIR / "boxplot_clusters_labeled.png")
    plt.close()
    
    print(f"    ✅ Charts saved to: {OUTPUT_DIR}")

if __name__ == "__main__":
    run_clustering_and_stats()

🤖 Running K-Means Clustering & Statistical Reporting...


D:\Geospatial_Workspace\geo310\lib\site-packages\sklearn\cluster\_kmeans.py:1419: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=2.
  warnings.warn(


    ✅ Clustered data saved to: results\cluster\tracts_clustered.geojson


C:\Users\14175\AppData\Local\Temp\ipykernel_6864\1608935071.py:116: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.boxplot(
D:\Geospatial_Workspace\geo310\lib\site-packages\seaborn\categorical.py:700: PendingDeprecationWarning: vert: bool will be deprecated in a future version. Use orientation: {'vertical', 'horizontal'} instead.
  artists = ax.bxp(**boxplot_kws)
D:\Geospatial_Workspace\geo310\lib\site-packages\seaborn\categorical.py:700: PendingDeprecationWarning: vert: bool will be deprecated in a future version. Use orientation: {'vertical', 'horizontal'} instead.
  artists = ax.bxp(**boxplot_kws)
D:\Geospatial_Workspace\geo310\lib\site-packages\seaborn\categorical.py:700: PendingDeprecationWarning: vert: bool will be deprecated in a future version. Use orientation: {'vertical', 'horizontal'} instead.
  artists = ax.bxp(**boxplot_kws)
D:\Geosp

    ✅ Charts saved to: results\cluster


In [33]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import geopandas as gpd
from pathlib import Path

# ---------------- CONFIGURATION ----------------
BASE_DIR = Path(".")
INPUT_FILE = BASE_DIR / "results/cluster/tracts_clustered_labeled.geojson" # Uses the file we just made
OUTPUT_DIR = BASE_DIR / "results/cluster"

# Set style
sns.set_theme(style="whitegrid")
plt.rcParams['figure.dpi'] = 300

def generate_svi_tertile_boxplot():
    print("📊 Generating SVI Tertile Boxplot...")

    # 1. Load Data
    if not INPUT_FILE.exists():
        print(f"❌ Input file not found: {INPUT_FILE}")
        return
    
    gdf = gpd.read_file(INPUT_FILE)
    
    # 2. Create 3 Groups (Tertiles) based on SVI
    # We use quantiles (33% and 66%) to ensure equal sample sizes in each group
    low_cutoff = gdf['SVI_Clean'].quantile(0.33)
    high_cutoff = gdf['SVI_Clean'].quantile(0.66)

    def classify_svi(svi_score):
        if svi_score <= low_cutoff:
            return "Low Vulnerability"
        elif svi_score <= high_cutoff:
            return "Medium Vulnerability"
        else:
            return "High Vulnerability"

    gdf['SVI_Group'] = gdf['SVI_Clean'].apply(classify_svi)

    # 3. Define Order and Colors
    # Order for the X-axis
    group_order = ["Low Vulnerability", "Medium Vulnerability", "High Vulnerability"]
    
    # Sequential Palette: Light Blue -> Medium Blue -> Dark Blue
    # This visually communicates "Increasing Intensity"
    tertile_palette = {
        "Low Vulnerability": "#a6cee3",    # Light Blue
        "Medium Vulnerability": "#1f78b4", # Medium Blue
        "High Vulnerability": "#08306b"    # Dark Navy
    }

    # 4. Generate Plot
    plt.figure(figsize=(10, 7))
    
    # Create the boxplot
    ax = sns.boxplot(
        x='SVI_Group',
        y='access_all',
        data=gdf,
        order=group_order,      # Force correct order
        palette=tertile_palette,
        width=0.5,              # Makes boxes slightly thinner (elegant look)
        linewidth=1.5           # Thicker lines for the "frame" look
    )

    # 5. Styling
    plt.title("Accessibility Distribution by Social Vulnerability Level", fontsize=14, pad=15)
    plt.xlabel("Social Vulnerability Group (Tertiles)", fontsize=12)
    plt.ylabel("Accessibility Score (E2SFCA)", fontsize=12)
    
    # Add grid explicitly on Y axis for readability
    ax.yaxis.grid(True, linestyle='--', alpha=0.7)
    ax.xaxis.grid(False) # Turn off x-grid for cleaner look

    # Save
    out_path = OUTPUT_DIR / "boxplot_svi_tertiles.png"
    plt.tight_layout()
    plt.savefig(out_path)
    plt.close()
    
    print(f"    ✅ SVI Tertile Boxplot saved to: {out_path}")

if __name__ == "__main__":
    generate_svi_tertile_boxplot()

📊 Generating SVI Tertile Boxplot...


C:\Users\14175\AppData\Local\Temp\ipykernel_6864\423683479.py:57: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  ax = sns.boxplot(
D:\Geospatial_Workspace\geo310\lib\site-packages\seaborn\categorical.py:700: PendingDeprecationWarning: vert: bool will be deprecated in a future version. Use orientation: {'vertical', 'horizontal'} instead.
  artists = ax.bxp(**boxplot_kws)
D:\Geospatial_Workspace\geo310\lib\site-packages\seaborn\categorical.py:700: PendingDeprecationWarning: vert: bool will be deprecated in a future version. Use orientation: {'vertical', 'horizontal'} instead.
  artists = ax.bxp(**boxplot_kws)
D:\Geospatial_Workspace\geo310\lib\site-packages\seaborn\categorical.py:700: PendingDeprecationWarning: vert: bool will be deprecated in a future version. Use orientation: {'vertical', 'horizontal'} instead.
  artists = ax.bxp(**boxplot_kws)


    ✅ SVI Tertile Boxplot saved to: results\cluster\boxplot_svi_tertiles.png


In [34]:
# ---------------------------------------------------------
# MAP 3: ULTIMATE LEGEND FIX (Force-Render Mode)
# ---------------------------------------------------------
def run_ultimate_legend_fix(gdf):
    print("🔥 Generating Map with FORCED LEGEND...")
    
    # Reproject
    gdf_web = gdf.to_crs(epsg=3857)
    
    # --- 1. DATA PREP ---
    if 'RPL_THEMES' in gdf_web.columns:
        gdf_web['SVI'] = gdf_web['RPL_THEMES'].replace(-999, np.nan)
    else:
        gdf_web['SVI'] = np.nan
    gdf_web['SVI'].fillna(gdf_web['SVI'].median(), inplace=True)

    time_col = 'nearest_time'
    if time_col not in gdf_web.columns:
        time_col = find_time_column(gdf_web)
    
    max_time = gdf_web[time_col].max()
    if pd.isna(max_time): max_time = 60
    gdf_web[time_col].fillna(max_time, inplace=True)

    # --- 2. THRESHOLDS ---
    svi_threshold = gdf_web['SVI'].median()
    time_threshold = gdf_web[time_col].median()

    # --- 3. CATEGORIES ---
    def get_category(row):
        is_high_svi = row['SVI'] >= svi_threshold
        is_high_acc = row[time_col] < time_threshold # Low time = High Access
        
        if is_high_svi and is_high_acc:
            return "High SVI / High Access"
        elif is_high_svi and not is_high_acc:
            return "High SVI / Low Access"
        elif not is_high_svi and is_high_acc:
            return "Low SVI / High Access"
        else:
            return "Low SVI / Low Access"

    gdf_web['category_label'] = gdf_web.apply(get_category, axis=1)

    # --- 4. COLORS (Professional PiYG) ---
    custom_colors = {
        "High SVI / Low Access":  "#e95e4f", # Magenta (Critical)
        "High SVI / High Access": "#48a2de", # Pink
        "Low SVI / High Access":  "#43d17f", # Dark Green (Ideal)
        "Low SVI / Low Access":   "#f4a629"  # Light Green
    }
    
    # --- 5. PLOTTING ---
    fig, ax = plt.subplots(figsize=(12, 12)) 
    
    # A. Plot the actual map data
    for cat, color in custom_colors.items():
        subset = gdf_web[gdf_web['category_label'] == cat]
        if len(subset) > 0:
            subset.plot(ax=ax, color=color, edgecolor='white', linewidth=0.05)
    
    # B. FORCE THE LEGEND (The "Invisible Dot" Trick)
    # We create empty scatter points for EVERY category just for the legend
    # This guarantees the legend box is full, even if data is missing or weird.
    legend_elements = []
    for cat, color in custom_colors.items():
        legend_elements.append(
            plt.Line2D([0], [0], marker='s', color='w', label=cat,
                       markerfacecolor=color, markersize=12)
        )

    add_basemap(ax, source=ctx.providers.CartoDB.Positron)
    add_north_arrow(ax)
    add_scale_bar(ax)
    
    ax.set_title("Neighborhood Typologies: Vulnerability vs. Access\n(Bivariate Classification)", 
                 fontweight='bold', fontsize=18)
    ax.axis('off')
    
    # --- C. RENDER LEGEND (Top Left - Safe Zone) ---
    ax.legend(
        handles=legend_elements,
        loc='upper left',          # Safer than outside
        bbox_to_anchor=(0.02, 0.98), # Fine tune inside the box
        title="Neighborhood Type",
        title_fontsize=12,
        fontsize=10,
        frameon=True,
        facecolor='white',
        edgecolor='black',
        framealpha=1
    )
    
    outfile = OUTPUT_DIR / "03_typologies_legend_forced.png"
    plt.savefig(outfile, bbox_inches='tight', dpi=300)
    plt.close()
    print(f"   ✅ Map SAVED (Legend Guaranteed) to: {outfile}")

# --- EXECUTE ---
if __name__ == "__main__":
    if INPUT_FILE.exists():
        gdf = gpd.read_file(INPUT_FILE)
        run_ultimate_legend_fix(gdf)
        print(f"\n🎉 Done! Check output in: {OUTPUT_DIR}")
    else:
        print("❌ Error: Input file not found.")

🔥 Generating Map with FORCED LEGEND...


C:\Users\14175\AppData\Local\Temp\ipykernel_6864\62653639.py:15: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  gdf_web['SVI'].fillna(gdf_web['SVI'].median(), inplace=True)


NameError: name 'find_time_column' is not defined

In [14]:
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import contextily as ctx
import osmnx as ox
import networkx as nx
from matplotlib_scalebar.scalebar import ScaleBar
from pathlib import Path
import warnings

# Suppress warnings
warnings.filterwarnings('ignore')

# ---------------------------------------------------------
# 1. CONFIGURATION
# ---------------------------------------------------------
BASE_DIR = Path(".")
DATA_PROCESSED = BASE_DIR / "data" / "processed"
DATA_RAW = BASE_DIR / "data" / "raw"
OUTPUT_DIR = BASE_DIR / "project_outputs_optimized"
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

TARGET_CRS = "EPSG:32129"
plt.rcParams.update({"font.family": "sans-serif", "figure.dpi": 300})

# ---------------------------------------------------------
# 2. YOUR ORIGINAL DATA GENERATION LOGIC (Restored)
# ---------------------------------------------------------
def load_data_optimized():
    print("📂 Loading Data...")
    tracts_path = DATA_PROCESSED / "tracts_pop_svi_projected.geojson"
    if not tracts_path.exists():
        tracts_path = DATA_PROCESSED / "tracts_svi_projected.geojson"
    
    gdf_tracts = gpd.read_file(tracts_path).to_crs(TARGET_CRS)
    
    buffer_fac_path = DATA_PROCESSED / "facilities_with_buffer.geojson"
    if not buffer_fac_path.exists():
        buffer_fac_path = DATA_PROCESSED / "facilities_projected.geojson"
        
    gdf_facilities = gpd.read_file(buffer_fac_path).to_crs(TARGET_CRS)
    
    # Ensure columns exist
    if 'BEDS' not in gdf_facilities.columns: gdf_facilities['BEDS'] = 100
    if 'population' not in gdf_tracts.columns: gdf_tracts['population'] = 1000
        
    return gdf_tracts, gdf_facilities

def calculate_od_matrix(G, tracts, facilities):
    print("⏱️  Calculating OD Matrix (Dijkstra)...")
    orig_nodes = ox.nearest_nodes(G, X=tracts.geometry.centroid.x, Y=tracts.geometry.centroid.y)
    dest_nodes = ox.nearest_nodes(G, X=facilities.geometry.x, Y=facilities.geometry.y)
    
    unique_origs = list(set(orig_nodes))
    od_times = {}
    
    # Cutoff 45 min
    for o_node in unique_origs:
        try:
            lengths = nx.single_source_dijkstra_path_length(G, o_node, weight='travel_time', cutoff=45*60)
            od_times[o_node] = lengths
        except:
            pass
            
    return orig_nodes, dest_nodes, od_times

def run_2sfca_sensitivity(tracts, facilities, orig_nodes, dest_nodes, od_times):
    print("🧮 Running 2SFCA Sensitivity (15min vs 30min)...")
    df_res = tracts.copy()
    
    for time_cutoff in [15, 30]:
        print(f"   ...Processing {time_cutoff} min Catchment")
        col_name = f'access_{time_cutoff}min'
        
        # Step 1: R_j
        r_j_list = []
        for j, fac in facilities.iterrows():
            f_node = dest_nodes[j]
            cap = fac['BEDS']
            served_pop = 0
            
            for i, tract in tracts.iterrows():
                t_node = orig_nodes[i]
                if t_node in od_times and f_node in od_times[t_node]:
                    time_min = od_times[t_node][f_node] / 60
                    if time_min <= time_cutoff:
                        served_pop += tract['population']
            
            r_j_list.append(cap / served_pop if served_pop > 0 else 0)
        
        facilities[f'R_{time_cutoff}'] = r_j_list
        
        # Step 2: A_i
        a_i_list = []
        for i, tract in tracts.iterrows():
            t_node = orig_nodes[i]
            sum_r = 0
            for j, fac in facilities.iterrows():
                f_node = dest_nodes[j]
                if t_node in od_times and f_node in od_times[t_node]:
                    time_min = od_times[t_node][f_node] / 60
                    if time_min <= time_cutoff:
                        sum_r += fac[f'R_{time_cutoff}']
            a_i_list.append(sum_r * 1000)
            
        df_res[col_name] = a_i_list
        
    return df_res

# ---------------------------------------------------------
# 3. PROFESSIONAL PLOTTING LOGIC (Applied to Memory Data)
# ---------------------------------------------------------
def plot_professional_map(gdf):
    print("🎨 Generating Professional Map...")
    
    # Reproject
    gdf_web = gdf.to_crs(epsg=3857)
    
    # Calculate Diff
    gdf_web['access_diff'] = gdf_web['access_30min'] - gdf_web['access_15min']
    
    # DEBUG PRINT
    print(f"   📊 Min Diff: {gdf_web['access_diff'].min():.4f}")
    print(f"   📊 Max Diff: {gdf_web['access_diff'].max():.4f}")
    
    fig, ax = plt.subplots(figsize=(12, 12))
    
    # Plot Magma
    try:
        gdf_web.plot(
            column='access_diff',
            ax=ax,
            cmap='magma',
            scheme='quantiles',
            k=5,
            alpha=0.9,
            edgecolor='white',
            linewidth=0.05
        )
    except:
        gdf_web.plot(column='access_diff', ax=ax, cmap='magma', alpha=0.9)

    # Styling
    try:
        ctx.add_basemap(ax, crs=3857, source=ctx.providers.CartoDB.Positron, alpha=0.6)
    except: pass

    # North Arrow
    x, y, arrow_length = 0.05, 0.95, 0.1
    ax.annotate('N', xy=(x, y), xytext=(x, y-arrow_length),
                arrowprops=dict(facecolor='black', width=5, headwidth=15),
                ha='center', va='center', fontsize=12, xycoords=ax.transAxes)

    # Scale Bar
    ax.add_artist(ScaleBar(1, location='lower left', box_alpha=0.5))

    ax.set_title("Sensitivity Analysis: Access Gain\n(Impact of Increasing Catchment from 15 to 30 min)", 
                 fontweight='bold', fontsize=16)
    ax.axis('off')

    # Force Legend
    import matplotlib.lines as mlines
    cmap = plt.cm.magma
    colors = [cmap(i) for i in [0.0, 0.25, 0.5, 0.75, 1.0]]
    labels = ["Low Gain", "Mod-Low", "Moderate", "Mod-High", "High Gain"]
    handles = [mlines.Line2D([], [], color=c, marker='s', linestyle='None', 
                            markersize=10, label=l) for c, l in zip(colors, labels)]

    ax.legend(handles=handles, loc='lower right', title="Score Increase", 
              frameon=True, framealpha=1, edgecolor='black')

    out_file = OUTPUT_DIR / "04_sensitivity_difference_professional.png"
    plt.savefig(out_file, bbox_inches='tight', dpi=300)
    plt.close()
    print(f"   ✅ Map saved to: {out_file}")

# ---------------------------------------------------------
# 4. MAIN EXECUTION
# ---------------------------------------------------------
if __name__ == "__main__":
    # 1. Load Geometries
    gdf_tracts, gdf_facilities = load_data_optimized()
    
    # 2. Load Network
    graph_path = DATA_RAW / "philly_drive_network.graphml"
    if graph_path.exists():
        print("🕸️  Loading Graph...")
        G = ox.load_graphml(graph_path)
        G = ox.project_graph(G, to_crs=TARGET_CRS)
        G = ox.add_edge_speeds(G)
        G = ox.add_edge_travel_times(G)
        
        # 3. Calculate Logic (Using your original functions)
        orig_nodes, dest_nodes, od_times = calculate_od_matrix(G, gdf_tracts, gdf_facilities)
        final_gdf = run_2sfca_sensitivity(gdf_tracts, gdf_facilities, orig_nodes, dest_nodes, od_times)
        
        # 4. Plot Directly
        plot_professional_map(final_gdf)
    else:
        print("❌ Error: Graph file missing. Cannot run calculation.")

📂 Loading Data...
🕸️  Loading Graph...
⏱️  Calculating OD Matrix (Dijkstra)...
🧮 Running 2SFCA Sensitivity (15min vs 30min)...
   ...Processing 15 min Catchment
   ...Processing 30 min Catchment
🎨 Generating Professional Map...
   📊 Min Diff: nan
   📊 Max Diff: nan
   ✅ Map saved to: project_outputs_optimized\04_sensitivity_difference_professional.png


In [1]:
import panel as pn
import geopandas as gpd
import hvplot.pandas
import pandas as pd
import numpy as np
from pathlib import Path

# Initialize Panel extension
pn.extension('bokeh')

# =========================================================
# 1. FILE PATH CONFIGURATION
# =========================================================
# Define base paths based on your provided structure
BASE_DIR = Path(".")

# Image Paths
# Ensure these match your folder structure exactly
IMG_EDGE_EFFECT = "results/01_missteps_edge_effects/01_error_edge_effect_demonstration_fixed.png"
IMG_POP = "results/02_intermediate_process/02_process_pop_density.png"
IMG_SVI = "results/02_intermediate_process/02_process_SVI_distribution.png"
IMG_STATIC_MAP = "results/03_final_deliverables/03_final_equity_gap_map_optimized.png"
IMG_SCATTER = "results/cluster/scatter_high_contrast.png"
IMG_BOXPLOT = "results/cluster/boxplot_high_contrast.png"
# Note: Ensure the folder 'final_deliverables' exists in your root, or adjust this path
IMG_LEGEND = "final_deliverables/publication_maps_v5/03_typologies_legend_forced.png"

# Data Path
DATA_PATH = "results/final_E2SFCA_results.geojson"

# =========================================================
# 2. DATA LOADING AND PREPROCESSING
# =========================================================
print("Loading data for interactive map...")
gdf = gpd.read_file(DATA_PATH).to_crs("EPSG:4326")

# Clean SVI Data: Replace -999 with NaN for transparency
if 'RPL_THEMES' in gdf.columns:
    gdf['SVI'] = gdf['RPL_THEMES'].replace(-999, np.nan)
else:
    gdf['SVI'] = np.nan

# Calculate Equity Gap (% Access Lost)
# Formula: (Access All - Access Medicaid) / Access All
gdf['Equity_Gap'] = (
    ((gdf['access_all'] - gdf['access_medicaid']) / gdf['access_all'].replace(0, 0.0001)) * 100
).clip(0, 100).fillna(0)

# Calculate Priority Index (Weighted Impact)
# Highlights areas that are both vulnerable AND losing access
gdf['Weighted_Impact'] = gdf['Equity_Gap'] * gdf['SVI'].fillna(0)

# Create a copy of the Medicaid access score for visualization
gdf['Access_Score'] = gdf['access_medicaid']

# Dynamic Scaling for Color Bars (95th percentile to handle outliers)
gap_max = gdf['Equity_Gap'].quantile(0.95)
if gap_max < 1: gap_max = 1.0
impact_max = gdf['Weighted_Impact'].quantile(0.95)

# =========================================================
# 3. INTERACTIVE MAP LOGIC
# =========================================================
def get_interactive_map(metric):
    # Configuration for each layer
    configs = {
        'SVI': {
            'cmap': 'OrRd', 
            'title': 'Social Vulnerability Index (SVI)', 
            'clim': (0, 1)
        },
        'Equity_Gap': {
            'cmap': 'Plasma', 
            'title': 'Equity Gap (% Access Lost due to Insurance)', 
            'clim': (0, gap_max)
        },
        'Weighted_Impact': {
            'cmap': 'Inferno', 
            'title': 'Priority Index (Gap x SVI)', 
            'clim': (0, impact_max)
        },
        'Access_Score': {
            'cmap': 'Viridis', 
            'title': 'Raw Accessibility Score (E2SFCA)', 
            'clim': None
        }
    }
    
    c = configs[metric]
    clim_args = {'clim': c['clim']} if c['clim'] else {}
    
    # Generate the map using hvplot
    return gdf.hvplot.polygons(
        c=metric, 
        cmap=c['cmap'], 
        title=c['title'],
        geo=True, 
        tiles='CartoLight', 
        alpha=0.7,
        hover_cols=['GEOID', 'SVI', 'Equity_Gap', 'Weighted_Impact', 'Access_Score'],
        line_width=0.05, 
        line_color='white',
        frame_height=600, 
        aspect='equal',
        colorbar=True,
        **clim_args
    )

# Create the Dropdown Widget
select_widget = pn.widgets.Select(
    name='Select Analytical Layer:', 
    options=['Equity_Gap', 'Weighted_Impact', 'SVI', 'Access_Score']
)

# Bind the function to the widget
interactive_map_pane = pn.Column(
    select_widget, 
    pn.bind(get_interactive_map, metric=select_widget)
)

# =========================================================
# 4. PROJECT NARRATIVE & ANALYSIS (EXPANDED)
# =========================================================

# CSS Styles for a professional academic look
custom_style = {
    'font-family': 'Helvetica, Arial, sans-serif',
    'max-width': '1000px',
    'margin': '0 auto',
    'padding': '20px',
    'line-height': '1.6',
    'color': '#333333'
}

header_style = {
    'color': '#2c3e50',
    'border-bottom': '2px solid #eee',
    'padding-bottom': '10px'
}

# --- Section 1: Introduction ---
text_intro = """
# Philadelphia Healthcare Equity Analysis
### Beyond Physical Distance: Measuring the Medicaid Access Gap

**Executive Summary**
Access to healthcare is often measured by physical proximity, but for low-income populations dependent on Medicaid, "accessibility" is legally and financially restricted. This project utilizes the **Enhanced Two-Step Floating Catchment Area (E2SFCA)** method to quantify the spatial disparity between *theoretical access* (all hospitals) and *actual access* (Medicaid-accepting facilities) in Philadelphia.

**Research Question:**
*How does limiting healthcare supply to Medicaid-accepting facilities alter the accessibility landscape for Philadelphia's most vulnerable neighborhoods?*
"""

# --- Section 2: Methodology ---
text_method = """
## 1. Methodology & Data Sources

To ensure a rigorous spatial analysis, this project integrated three distinct datasets:
1.  **Supply:** Hospital locations from OpenDataPhilly, cross-referenced with CMS General Information data to verify Medicaid acceptance status.
2.  **Demand:** Population data from the 2021 ACS (Census) and Social Vulnerability Index (SVI) scores from the CDC.
3.  **Network:** A multimodal driving network constructed using OpenStreetMap (OSM) via the OSMnx Python library.

**The Edge Effect Challenge**
A common methodological flaw in urban spatial analysis is the "Edge Effect," where facilities immediately outside the city boundaries are ignored. This artificially deflates accessibility scores for border neighborhoods. As shown below, we corrected this by buffering the study area to include facilities in New Jersey and the Pennsylvania suburbs.
"""

text_edge_desc = """
*Figure 1: The "Edge Effect" Correction. Red markers indicate facilities outside the city limits (e.g., in Camden, NJ or Lower Merion, PA) that were included in the calculation to ensure accurate scoring for peripheral Philadelphia neighborhoods.*
"""

# --- Section 3: Input Variables ---
text_inputs = """
## 2. Input Variables
The analysis considers the interaction between demand (Population Density) and sociodemographic risk (Social Vulnerability).
"""

# --- Section 4: Results Analysis ---
text_results = """
## 3. Results: The Equity Gap
The map below visualizes the **Equity Gap**, defined as the percentage of accessibility lost when a patient is restricted to Medicaid-only facilities.

**Key Analytical Findings:**
1.  **The Central Paradox:** While Center City possesses the highest density of medical infrastructure, the Medicaid acceptance rate varies. However, the sheer volume of providers buffers this area from becoming a desert.
2.  **The North Philadelphia Crisis:** Areas with the highest Social Vulnerability (High SVI) coincides with significant "Access Loss." This suggests that the populations most in need of safety-net care face structural barriers even in urban settings.
3.  **Structural Inequality:** The correlation between high SVI and high Equity Gap suggests that the healthcare market is not optimized for public insurance users in specific residential zones.
"""

# --- Section 5: Neighborhood Typologies ---
text_cluster = """
## 4. Neighborhood Typologies (Cluster Analysis)
To move beyond simple mapping, we performed **K-Means Clustering** to categorize neighborhoods based on the intersection of Accessibility and Vulnerability. This reveals four distinct neighborhood types:

1.  **Privileged (Low SVI, High Access):** Neighborhoods with high resources and high healthcare availability.
2.  **Supported (High SVI, High Access):** Vulnerable neighborhoods that are well-served by safety-net institutions.
3.  **Critical Deserts (High SVI, Low Access):** The highest priority for policy intervention. These residents face high socioeconomic risks and have the lowest physical access to care.
4.  **Moderate Access:** Transitional zones.
"""

# --- Section 6: Policy Implications & Conclusion ---
text_policy = """
## 5. Policy Implications & Conclusion
Based on the "Weighted Impact" metric, which combines the Equity Gap with Social Vulnerability, we recommend the following interventions:

* **Targeted Expansion:** New Federally Qualified Health Centers (FQHCs) should be prioritized in the "Critical Desert" clusters identified in North and Southwest Philadelphia.
* **Transportation Subsidies:** For high-gap areas, improving transit connectivity to the existing safety-net hospitals is more cost-effective than building new infrastructure.
* **Insurance Incentives:** State-level policy should explore incentives for private hospitals in "High Gap" zones to increase their Medicaid intake caps.

**Limitations**
This study assumes travel by car. Future iterations should incorporate public transit network analysis (GTFS data) to better represent the reality of low-income residents who are less likely to own private vehicles.
"""

text_interactive_header = """
## 6. Interactive Data Explorer
Use the dashboard below to toggle between different analytical layers.
"""

# =========================================================
# 5. DASHBOARD ASSEMBLY
# =========================================================
print("Assembling final dashboard layout...")

# Create the main layout container
dashboard = pn.Column(
    # --- Header ---
    pn.pane.Markdown(text_intro, styles=custom_style),
    pn.layout.Divider(),
    
    # --- Methodology ---
    pn.pane.Markdown(text_method, styles=custom_style),
    pn.pane.PNG(IMG_EDGE_EFFECT, width=800, align='center'),
    pn.pane.Markdown(text_edge_desc, styles={'font-size': '0.9em', 'font-style': 'italic', 'color': '#666', 'text-align': 'center'}),
    
    # --- Inputs ---
    pn.pane.Markdown(text_inputs, styles=custom_style),
    pn.Row(
        pn.pane.PNG(IMG_POP, width=450),
        pn.pane.PNG(IMG_SVI, width=450),
        align='center'
    ),
    
    # --- Results ---
    pn.layout.Divider(),
    pn.pane.Markdown(text_results, styles=custom_style),
    pn.pane.PNG(IMG_STATIC_MAP, width=800, align='center'),
    
    # --- Clustering ---
    pn.layout.Divider(),
    pn.pane.Markdown(text_cluster, styles=custom_style),
    pn.Row(
        pn.pane.PNG(IMG_LEGEND, width=400),
        pn.pane.PNG(IMG_BOXPLOT, width=500),
        align='center'
    ),
    pn.pane.PNG(IMG_SCATTER, width=800, align='center'),
    
    # --- Policy & Conclusion ---
    pn.layout.Divider(),
    pn.pane.Markdown(text_policy, styles=custom_style),
    
    # --- Interactive Map ---
    pn.layout.Divider(),
    pn.pane.Markdown(text_interactive_header, styles=custom_style),
    interactive_map_pane,
    
    # --- Footer ---
    pn.layout.Divider(),
    pn.pane.Markdown(
        "**Data Sources:** OpenDataPhilly, US Census Bureau ACS 2021, CMS Hospital Data. | **Author:** CPLN 6720 Student", 
        styles={'text-align': 'center', 'color': 'gray', 'font-size': '0.8em', 'margin-bottom': '50px'}
    )
)

# =========================================================
# 6. EXPORT TO HTML
# =========================================================
# Ensure output directory exists
output_dir = Path("docs")
output_dir.mkdir(parents=True, exist_ok=True)
output_file = output_dir / "index.html"

print(f"Exporting website to {output_file}...")

# 'embed=True' ensures all images are packed inside the HTML file
# 'resources=INLINE' ensures JS libraries are included, making it standalone
dashboard.save(str(output_file), embed=True, resources='INLINE')

print("SUCCESS: The website has been generated in the 'docs' folder.")

Loading data for interactive map...
Assembling final dashboard layout...
Exporting website to docs\index.html...
                                                                                                                       

SUCCESS: The website has been generated in the 'docs' folder.
